# Utilities

> Utility functions for configuration loading, logging setup, etc.

In [ ]:
#| default_exp utils

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import yaml
from pathlib import Path
from fastcore.utils import *
import wandb
import os
import datetime

## Configuration Loading

In [ ]:
#| export
def load_config(config_path: str|Path):
    """Loads configuration settings from a YAML file.

    Args:
        config_path: Path to the YAML configuration file.

    Returns:
        A dictionary containing the configuration settings.

    Raises:
        FileNotFoundError: If the config file doesn't exist.
        yaml.YAMLError: If there's an error parsing the YAML file.
    """
    config_path = Path(config_path)
    if not config_path.is_file():
        raise FileNotFoundError(f"Configuration file not found at {config_path}")
    try:
        with open(config_path, 'r') as f:
            config = yaml.safe_load(f)
        return config
    except yaml.YAMLError as e:
        print(f"Error parsing YAML file: {config_path}")
        raise e

In [ ]:
show_doc(load_config)

---

### load_config

>      load_config (config_path:str|pathlib.Path)

*Loads configuration settings from a YAML file.

Args:
    config_path: Path to the YAML configuration file.

Returns:
    A dictionary containing the configuration settings.

Raises:
    FileNotFoundError: If the config file doesn't exist.
    yaml.YAMLError: If there's an error parsing the YAML file.*

### Usage Example (Loading)

In [ ]:
#|eval: false
# Create a dummy config for testing if it doesn't exist
config_dir = Path('configs')
config_dir.mkdir(exist_ok=True)
dummy_config_path = config_dir / 'config.yaml'

if not dummy_config_path.exists():
    dummy_config_content = """
    project_name: Adaptive_Patching_VIT_fastai
    paths:
      data_base: /path/to/datasets
      output_dir: /path/to/outputs
    model:
      llm_name_or_path: lmsys/vicuna-7b-v1.5
      vision_encoder_name_or_path: openai/clip-vit-large-patch14-336
    logging:
      wandb:
        enabled: true
        project: adaptive_patching_vit_test
        entity: your_wandb_entity # Replace with your entity
    """
    with open(dummy_config_path, 'w') as f:
        f.write(dummy_config_content)
    print(f"Created dummy config: {dummy_config_path}")

# Load the config
try:
    config = load_config(dummy_config_path)
    print("\nConfig loaded successfully:")
    print(config)
except Exception as e:
    print(f"Error loading config: {e}")


Created dummy config: configs/config.yaml

Config loaded successfully:
{'project_name': 'Adaptive_Patching_VIT_fastai', 'paths': {'data_base': '/path/to/datasets', 'output_dir': '/path/to/outputs'}, 'model': {'llm_name_or_path': 'lmsys/vicuna-7b-v1.5', 'vision_encoder_name_or_path': 'openai/clip-vit-large-patch14-336'}, 'logging': {'wandb': {'enabled': True, 'project': 'adaptive_patching_vit_test', 'entity': 'your_wandb_entity'}}}


## Weights & Biases Integration

In [ ]:
#| export
def init_wandb(config: dict, job_type: str = "training", run_name: str | None = None):
    """Initializes a Weights & Biases run.

    Args:
        config: The configuration dictionary, expected to have a 'logging.wandb' section.
        job_type: Type of the job (e.g., 'training', 'evaluation').
        run_name: Optional name for the run. Defaults to a timestamped name.

    Returns:
        The initialized wandb run object, or None if W&B is disabled.

    Notes:
        Requires WANDB_API_KEY environment variable or prior `wandb login`.
        Reads 'project' and 'entity' from config['logging']['wandb'].
    """
    wandb_config = config.get('logging', {}).get('wandb', {})
    if not wandb_config.get('enabled', False):
        print("W&B logging is disabled in the configuration.")
        return None

    project = wandb_config.get('project', 'default_project')
    entity = wandb_config.get('entity') # Can be None, W&B will use default

    if run_name is None:
        run_name = f"{job_type}_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}"

    try:
        run = wandb.init(
            project=project,
            entity=entity,
            config=config, # Log the entire config
            name=run_name,
            job_type=job_type,
            reinit=True # Allows re-initialization in notebooks
        )
        print(f"W&B run initialized: {run.name} (Project: {project}, Entity: {entity or 'default'})")
        print(f"Track run at: {run.url}")
        return run
    except Exception as e:
        print(f"Error initializing W&B: {e}")
        print("Ensure WANDB_API_KEY is set or you are logged in (`wandb login`).")
        return None

In [ ]:
show_doc(init_wandb)

---

### init_wandb

>      init_wandb (config:dict, job_type:str='training', run_name:str|None=None)

*Initializes a Weights & Biases run.

Args:
    config: The configuration dictionary, expected to have a 'logging.wandb' section.
    job_type: Type of the job (e.g., 'training', 'evaluation').
    run_name: Optional name for the run. Defaults to a timestamped name.

Returns:
    The initialized wandb run object, or None if W&B is disabled.

Notes:
    Requires WANDB_API_KEY environment variable or prior `wandb login`.
    Reads 'project' and 'entity' from config['logging']['wandb'].*

### Usage Example (W&B)

In [ ]:
#|eval: false
# Load config first
try:
    config_path = 'configs/config.yaml'
    config = load_config(config_path)
    print(f"Loaded config from {config_path}")

    # Initialize W&B (replace with your entity in config.yaml or set WANDB_ENTITY env var)
    # Make sure wandb logging is enabled in the config
    if config.get('logging', {}).get('wandb', {}).get('enabled'):
        wandb_run = init_wandb(config, job_type="test_initialization")

        if wandb_run:
            print("W&B run object received.")
            # Example of logging something
            wandb_run.log({"test_metric": 1.0})
            wandb_run.finish() # Finish the run when done
            print("W&B run finished.")
        else:
            print("Failed to initialize W&B run.")
    else:
        print("W&B is disabled in config, skipping initialization.")

except FileNotFoundError:
    print(f"Config file not found at {config_path}. Skipping W&B initialization test.")
except Exception as e:
    print(f"An error occurred: {e}")

wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Loaded config from configs/config.yaml


wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

  ········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: numb3r33 to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


Error initializing W&B: failed to upsert bucket: returned error 404: {"data":{"upsertBucket":null},"errors":[{"message":"entity your_wandb_entity not found during upsertBucket","path":["upsertBucket"]}]}
Ensure WANDB_API_KEY is set or you are logged in (`wandb login`).
Failed to initialize W&B run.


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()